In [1]:
import dotenv
dotenv.load_dotenv()

from chains.hospital_cypher_chain import hospital_cypher_chain

question = """What is the average visit duration for
emergency visits in North Carolina?"""
response = hospital_cypher_chain.invoke(question)

print(response.get("result"))



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (v:Visit)-[:AT]->(h:Hospital)
WHERE h.state_name = 'NC' AND v.admission_type = 'Emergency' AND v.status = 'DISCHARGED'
WITH v, duration.between(date(v.admission_date), date(v.discharge_date)).days AS visit_duration
RETURN AVG(visit_duration) AS average_visit_duration
Full Context:
[{'average_visit_duration': 15.072972972972991}]

> Finished chain.
The average visit duration for emergency visits in North Carolina is approximately 15.07 days.


In [2]:
question = """Which state had the largest percent increase
           in Medicaid visits from 2022 to 2023?"""
response = hospital_cypher_chain.invoke(question)
print(response.get("result"))



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (h:Hospital)<-[:AT]-(v:Visit)-[:COVERED_BY]->(p:Payer)
WHERE p.name = 'Medicaid' AND v.admission_date >= '2022-01-01' AND
v.admission_date < '2024-01-01'
WITH h.state_name AS state, COUNT(v) AS visit_count,
     SUM(CASE WHEN v.admission_date >= '2022-01-01' AND
     v.admission_date < '2023-01-01' THEN 1 ELSE 0 END) AS count_2022,
     SUM(CASE WHEN v.admission_date >= '2023-01-01' AND
     v.admission_date < '2024-01-01' THEN 1 ELSE 0 END) AS count_2023
WITH state, visit_count, count_2022, count_2023,
     (toFloat(count_2023) - toFloat(count_2022)) / toFloat(count_2022) * 100
     AS percent_increase
RETURN state, percent_increase
ORDER BY percent_increase DESC
LIMIT 1
Full Context:
[{'state': 'TX', 'percent_increase': 8.823529411764707}]

> Finished chain.
The state with the largest percent increase in Medicaid visits from 2022 to 2023 is Texas (TX) with a percent increase of 8.82%.


In [4]:

from agents.hospital_rag_agent import hospital_rag_agent_executor

response = hospital_rag_agent_executor.invoke(
    {"input": "What is the wait time at Wallace-Hamilton?"}
)

print(f'{response["input"]}=')
print(f'{response["output"]}=')
response["intermediate_steps"]



> Entering new AgentExecutor chain...

Invoking: `Waits` with `Wallace-Hamilton`


5 hours 59 minutesThe current wait time at Wallace-Hamilton is 5 hours and 59 minutes.

> Finished chain.
What is the wait time at Wallace-Hamilton?=
The current wait time at Wallace-Hamilton is 5 hours and 59 minutes.=


[(AgentActionMessageLog(tool='Waits', tool_input='Wallace-Hamilton', log='\nInvoking: `Waits` with `Wallace-Hamilton`\n\n\n', message_log=[AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"__arg1":"Wallace-Hamilton"}', 'name': 'Waits'}})]),
  '5 hours 59 minutes')]

In [5]:
# with this he tries a vector search/Experiences tool  
response = hospital_rag_agent_executor.invoke(
    {"input": "Show me reviews written by patient 7674"}
)

print(f'{response["input"]}=')
print(f'{response["output"]}=')
response["intermediate_steps"]



> Entering new AgentExecutor chain...

Invoking: `Experiences` with `Show me reviews written by patient 7674`


{'query': 'Show me reviews written by patient 7674', 'result': 'I\'m sorry, but there are no reviews provided by a patient with the identifier "7674" in the context I have been given. If you have any other questions or need information about a different patient, feel free to ask.'}I'm sorry, but there are no reviews provided by a patient with the identifier "7674" in the context I have been given. If you have any other questions or need information about a different patient, feel free to ask.

> Finished chain.
Show me reviews written by patient 7674=
I'm sorry, but there are no reviews provided by a patient with the identifier "7674" in the context I have been given. If you have any other questions or need information about a different patient, feel free to ask.=


[(AgentActionMessageLog(tool='Experiences', tool_input='Show me reviews written by patient 7674', log='\nInvoking: `Experiences` with `Show me reviews written by patient 7674`\n\n\n', message_log=[AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"__arg1":"Show me reviews written by patient 7674"}', 'name': 'Experiences'}})]),
  {'query': 'Show me reviews written by patient 7674',
   'result': 'I\'m sorry, but there are no reviews provided by a patient with the identifier "7674" in the context I have been given. If you have any other questions or need information about a different patient, feel free to ask.'})]

In [6]:
# with this he does a cypher query
response = hospital_rag_agent_executor.invoke(
    {"input": "Query the graph database to show me the reviews written by patient 7674"}
)

print(f'{response["input"]}=')
print(f'{response["output"]}=')
response["intermediate_steps"]



> Entering new AgentExecutor chain...

Invoking: `Graph` with `Show me the reviews written by patient 7674`




> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (p:Patient {id: 7674})-[:HAS]->(v:Visit)-[:WRITES]->(r:Review)
RETURN r.text AS review_written

Full Context:
[{'review_written': 'The hospital provided exceptional care, but the billing process was confusing and frustrating. Clearer communication about costs would have been appreciated.'}]

> Finished chain.
{'query': 'Show me the reviews written by patient 7674', 'result': 'Here is the review written by patient 7674: \n\n"The hospital provided exceptional care, but the billing process was confusing and frustrating. Clearer communication about costs would have been appreciated."'}Patient 7674 wrote the following review: "The hospital provided exceptional care, but the billing process was confusing and frustrating. Clearer communication about costs would have been appreciated."

> Finished chain.
Query the g

[(AgentActionMessageLog(tool='Graph', tool_input='Show me the reviews written by patient 7674', log='\nInvoking: `Graph` with `Show me the reviews written by patient 7674`\n\n\n', message_log=[AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"__arg1":"Show me the reviews written by patient 7674"}', 'name': 'Graph'}})]),
  {'query': 'Show me the reviews written by patient 7674',
   'result': 'Here is the review written by patient 7674: \n\n"The hospital provided exceptional care, but the billing process was confusing and frustrating. Clearer communication about costs would have been appreciated."'})]